# Выполнение практической работы № 2. (Сучков В.В. ББМО-01-22)
## Реализация Adversarial (DeepFool, FGSM) атаки на модели машинного обучения (Lenet, FC)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Подготовка репозитория для среды выполнения Colab

**Adversarial атака — алгоритм действий, целью которого является получение вектора, подающегося на вход алгоритму, на котором алгоритм выдает некорректный выход.**

**Целью такой атаки является подбор такого вектора шума, при наложении которого будут оказываться минимальные изменения по отношению к исходному вектору, но результат определения класса будет не верным**

**В отличие от FGSM DeepFool старается минимизировать шум, при этом сделать так, чтобы классификация была ошибочной.**

In [ ]:
# Загрузим репозиторий в среду выполнения
!cp -r /content/drive/MyDrive/Учеба/Семестр\ №\ 3/Анализ\ защищенности\ систем\ искусственного\ интеллекта/prz_2/* /content/
# Удалим лишние файлы
!rm  *.ipynb

Cloning into 'EEL6812_DeepFool_Project'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 96 (delta 2), reused 1 (delta 1), pack-reused 93
Receiving objects: 100% (96/96), 33.99 MiB | 18.07 MiB/s, done.
Resolving deltas: 100% (27/27), done.


# Ход работы
### Импортируем библиотеки


In [ ]:
import numpy as np
import os, torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision.transforms import transforms
from models.project_models import FC_500_150, LeNet_CIFAR, LeNet_MNIST, Net
from utils.project_utils import get_clip_bounds, model_train, model_eval, evaluate_attack

### Устанавливаем rand_seed и выбираем устройство выполнения


In [ ]:
# Устанавливаем случайное число
rand_seed = 18
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)

use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

### Загружаем датасет MNIST и предобрабатываем его


In [ ]:
# Загружаем датасет MNIST
mnist_mean = 0.5
mnist_std = 0.5
mnist_dim = 28

mnist_min, mnist_max = get_clip_bounds(mnist_mean,
                                       mnist_std,
                                       mnist_dim)
mnist_min = mnist_min.to(device)
mnist_max = mnist_max.to(device)

mnist_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=mnist_mean,
        std=mnist_std)])

mnist_tf_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=mnist_mean,
        std=mnist_std)])

mnist_tf_inv = transforms.Compose([
    transforms.Normalize(
        mean=0.0,
        std=np.divide(1.0, mnist_std)),
    transforms.Normalize(
        mean=np.multiply(-1.0, mnist_std),
        std=1.0)])

mnist_temp = datasets.MNIST(root='datasets/mnist', train=True,
                            download=True, transform=mnist_tf_train)
mnist_train, mnist_val = random_split(mnist_temp, [50000, 10000])

mnist_test = datasets.MNIST(root='datasets/mnist', train=False,
                            download=True, transform=mnist_tf)

100%|██████████| 9912422/9912422 [00:00<00:00, 29846981.64it/s]


Extracting datasets/mnist/MNIST/raw/train-images-idx3-ubyte.gz to datasets/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 26985006.42it/s]


Extracting datasets/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to datasets/mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 27945479.58it/s]


Extracting datasets/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to datasets/mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 20137979.67it/s]


Extracting datasets/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to datasets/mnist/MNIST/raw



### Загружаем датасет CIFAR-10 и предобрабатываем его


In [ ]:
# Загрузим датасет CIFAR-10
cifar_mean = [0.491, 0.482, 0.447]
cifar_std = [0.202, 0.199, 0.201]
cifar_dim = 32

cifar_min, cifar_max = get_clip_bounds(cifar_mean,
                                       cifar_std,
                                       cifar_dim)
cifar_min = cifar_min.to(device)
cifar_max = cifar_max.to(device)

cifar_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=cifar_mean,
        std=cifar_std)])

cifar_tf_train = transforms.Compose([
    transforms.RandomCrop(
        size=cifar_dim,
        padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=cifar_mean,
        std=cifar_std)])

cifar_tf_inv = transforms.Compose([
    transforms.Normalize(
        mean=[0.0, 0.0, 0.0],
        std=np.divide(1.0, cifar_std)),
    transforms.Normalize(
        mean=np.multiply(-1.0, cifar_mean),
        std=[1.0, 1.0, 1.0])])

cifar_temp = datasets.CIFAR10(root='datasets/cifar-10', train=True,
                              download=True, transform=cifar_tf_train)
cifar_train, cifar_val = random_split(cifar_temp, [40000, 10000])

cifar_test = datasets.CIFAR10(root='datasets/cifar-10', train=False,
                              download=True, transform=cifar_tf)
cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                'dog', 'frog', 'horse', 'ship', 'truck']

100%|██████████| 170498071/170498071 [00:02<00:00, 62804720.85it/s]


Extracting datasets/cifar-10/cifar-10-python.tar.gz to datasets/cifar-10
Files already downloaded and verified


### Настраиваем гиперпараметры

In [ ]:
# Установимм размер разбивки на 64 батча
batch_size = 64
workers = 4

mnist_loader_train = DataLoader(mnist_train, batch_size=batch_size,
                                shuffle=True, num_workers=workers)
mnist_loader_val = DataLoader(mnist_val, batch_size=batch_size,
                              shuffle=False, num_workers=workers)
mnist_loader_test = DataLoader(mnist_test, batch_size=batch_size,
                               shuffle=False, num_workers=workers)

cifar_loader_train = DataLoader(cifar_train, batch_size=batch_size,
                                shuffle=True, num_workers=workers)
cifar_loader_val = DataLoader(cifar_val, batch_size=batch_size,
                              shuffle=False, num_workers=workers)
cifar_loader_test = DataLoader(cifar_test, batch_size=batch_size,
                               shuffle=False, num_workers=workers)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# Устанавливаем параметры для обучения
train_model = True

epochs = 50
epochs_nin = 100

lr = 0.004
lr_nin = 0.01
lr_scale = 0.5

momentum = 0.9

print_step = 5

deep_batch_size = 10
deep_num_classes = 10
deep_overshoot = 0.02
deep_max_iters = 50

deep_args = [deep_batch_size, deep_num_classes,
             deep_overshoot, deep_max_iters]

if not os.path.isdir('weights/deepfool'):
    os.makedirs('weights/deepfool', exist_ok=True)

if not os.path.isdir('weights/fgsm'):
    os.makedirs('weights/fgsm', exist_ok=True)

## Реализация атак
### Реализация атаки FGSM на модель Lenet и датасет MNIST при помощи функций EEL6812


In [ ]:
fgsm_eps = 0.5
model = LeNet_MNIST().to(device)
model.load_state_dict(torch.load('weights/clean/mnist_lenet.pth'))

if train_model:
    opt = torch.optim.SGD(model.parameters(),
                          lr=lr * lr_scale,
                          momentum=momentum)
    _, _, _, _, = model_train(device, model, opt, epochs,
                              mnist_loader_train, mnist_loader_val,
                              print_step, mnist_min, mnist_max,
                              fgsm_eps, is_fgsm=True)
    torch.save(model.state_dict(), 'weights/fgsm/mnist_lenet.pth')

model.load_state_dict(torch.load('weights/fgsm/mnist_lenet.pth'))
_, _ = model_eval(device, model, mnist_loader_test,
    mnist_min, mnist_max, fgsm_eps, is_fgsm=True)
_, _ = model_eval(device, model, mnist_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch [1]
    Train Acc : 0.4597,  Train Loss : 1.5236
      Val Acc : 0.5082,    Val Loss : 1.3089
Epoch [5]
    Train Acc : 0.7321,  Train Loss : 0.7488
      Val Acc : 0.7398,    Val Loss : 0.7164
Epoch [10]
    Train Acc : 0.8570,  Train Loss : 0.4157
      Val Acc : 0.8677,    Val Loss : 0.3872
Epoch [15]
    Train Acc : 0.8948,  Train Loss : 0.3107
      Val Acc : 0.8992,    Val Loss : 0.3004
Epoch [20]
    Train Acc : 0.9101,  Train Loss : 0.2613
      Val Acc : 0.9089,    Val Loss : 0.2805
Epoch [25]
    Train Acc : 0.9165,  Train Loss : 0.2430
      Val Acc : 0.9107,    Val Loss : 0.2619
Epoch [30]
    Train Acc : 0.9293,  Train Loss : 0.2101
      Val Acc : 0.9134,    Val Loss : 0.2682
Epoch [35]
    Train Acc : 0.9343,  Train Loss : 0.1933
      Val Acc : 0.9626,    Val Loss : 0.1079
Epoch [40]
    Train Acc : 0.9312,  Train Loss : 0.2023
      Val Acc : 0.9240,    Val Loss : 0.2291
Epoch [45]
    Train Acc : 0.9443,  Train Loss : 0.1629
      Val Acc : 0.9342,    Val Loss :

**Функция потерь при атаке на модель Lenet увеличилась, и точность снизилась на 5%**
## Реализация атаки FGSM на модель FC и датасет MNIST при помощи функций EEL6812


In [ ]:
fgsm_eps = 0.2
model = FC_500_150().to(device)
model.load_state_dict(torch.load('weights/clean/mnist_fc.pth'))

if train_model:
    opt = torch.optim.SGD(model.parameters(),
                          lr=lr * lr_scale,
                          momentum=momentum)
    _, _, _, _, = model_train(device, model, opt, epochs,
                              mnist_loader_train, mnist_loader_val,
                              print_step, mnist_min, mnist_max,
                              fgsm_eps, is_fgsm=True)
    torch.save(model.state_dict(), 'weights/fgsm/mnist_fc.pth')

model.load_state_dict(torch.load('weights/fgsm/mnist_fc.pth'))
_, _ = model_eval(device, model, mnist_loader_test,
    mnist_min, mnist_max, fgsm_eps, is_fgsm=True)
_, _ = model_eval(device, model, mnist_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

Epoch [1]
    Train Acc : 0.5811,  Train Loss : 1.2243
      Val Acc : 0.6651,    Val Loss : 0.9472
Epoch [5]
    Train Acc : 0.7521,  Train Loss : 0.7041
      Val Acc : 0.7602,    Val Loss : 0.6845
Epoch [10]
    Train Acc : 0.7963,  Train Loss : 0.5866
      Val Acc : 0.7981,    Val Loss : 0.5899
Epoch [15]
    Train Acc : 0.8206,  Train Loss : 0.5214
      Val Acc : 0.8240,    Val Loss : 0.5371
Epoch [20]
    Train Acc : 0.8357,  Train Loss : 0.4780
      Val Acc : 0.8363,    Val Loss : 0.4940
Epoch [25]
    Train Acc : 0.8495,  Train Loss : 0.4428
      Val Acc : 0.8459,    Val Loss : 0.4688
Epoch [30]
    Train Acc : 0.8585,  Train Loss : 0.4137
      Val Acc : 0.8503,    Val Loss : 0.4563
Epoch [35]
    Train Acc : 0.8648,  Train Loss : 0.3940
      Val Acc : 0.8552,    Val Loss : 0.4402
Epoch [40]
    Train Acc : 0.8718,  Train Loss : 0.3750
      Val Acc : 0.8605,    Val Loss : 0.4265
Epoch [45]
    Train Acc : 0.8792,  Train Loss : 0.3568
      Val Acc : 0.8671,    Val Loss :

**Влияние FGSM на FC модель гораздо сильнее, точность уменьшилась на 11%**
## Реализация атаки DeepFool на модель Lenet и датасет CIFAR-10 при помощи функций EEL6812 для этого из функции меняем значение is_fgsm на False

In [ ]:
# Влияние FGSM на FC модель гораздо сильнее, точность уменьшилась на 11%
# Реализация атаки DeepFool на модель Lenet и датасет CIFAR-10 при помощи функций EEL6812 для этого из функции меняем значение is_fgsm на False
model = LeNet_CIFAR().to(device)
model.load_state_dict(torch.load('weights/clean/cifar_lenet.pth'))

if train_model:
    opt = torch.optim.SGD(model.parameters(),
                          lr=lr * lr_scale,
                          momentum=momentum)
    _, _, _, _, = model_train(device, model, opt, 5,
                              cifar_loader_train, cifar_loader_val,
                              1, cifar_min, cifar_max,
                              deep_args, is_fgsm=False)
    torch.save(model.state_dict(), 'weights/deepfool/cifar_lenet.pth')

model.load_state_dict(torch.load('weights/deepfool/cifar_lenet.pth'))
_, _ = model_eval(device, model, cifar_loader_test,
    cifar_min, cifar_max, deep_args, is_fgsm=False)
_, _ = model_eval(device, model, cifar_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

Epoch [1]
    Train Acc : 0.1658,  Train Loss : 1.1754
      Val Acc : 0.1631,    Val Loss : 1.2258
Epoch [2]
    Train Acc : 0.1721,  Train Loss : 1.2243
      Val Acc : 0.1756,    Val Loss : 1.2913
Epoch [3]
    Train Acc : 0.1731,  Train Loss : 1.2354
      Val Acc : 0.1687,    Val Loss : 1.2278
Epoch [4]
    Train Acc : 0.1738,  Train Loss : 1.2271
      Val Acc : 0.1713,    Val Loss : 1.2328
Epoch [5]
    Train Acc : 0.1712,  Train Loss : 1.2290
      Val Acc : 0.1684,    Val Loss : 1.2457
Evaluation (DeepFool Images)
     Test Acc : 0.1718,   Test Loss : 1.2422
Evaluation (Clean Images)
     Test Acc : 0.6599,   Test Loss : 1.2451


**Время выполнения атаки сильно больше чем при атаке FGSM.**

**Однако результативность DeepFool в разы выше, точность от атаки снизилась в 3 раза, при этом функция потерь примерно равна**

*Это объяняется тем, что модель старается подобрать минимальный шум, который при этом будем искажать оперделяемый класс*
### Проверка атаки на Lenet при помощи evaluate_attack

In [ ]:
fgsm_eps = 0.5
model = LeNet_MNIST().to(device)
model.load_state_dict(torch.load('weights/clean/mnist_lenet.pth', map_location=torch.device('cpu')))
evaluate_attack('mnist_lenet_fgsm.csv', 'results', device, model, mnist_loader_test, mnist_min, mnist_max,
fgsm_eps, is_fgsm=True)
print('')
evaluate_attack('mnist_lenet_deepfool.csv', 'results', device, model,
mnist_loader_test, mnist_min, mnist_max, deep_args, is_fgsm=False)
if device.type == 'cuda': torch.cuda.empty_cache()

FGSM Test Error : 87.89%
FGSM Robustness : 4.58e-01
FGSM Time (All Images) : 0.29 s
FGSM Time (Per Image) : 28.86 us

DeepFool Test Error : 98.74%
DeepFool Robustness : 9.64e-02
DeepFool Time (All Images) : 193.32 s
DeepFool Time (Per Image) : 19.33 ms


### Проверка атаки на FC при помощи evaluate_attack

In [ ]:
fgsm_eps = 0.2
model = FC_500_150().to(device)
model.load_state_dict(torch.load('weights/clean/mnist_fc.pth', map_location=torch.device('cpu')))
evaluate_attack('mnist_fc_fgsm.csv', 'results', device, model, mnist_loader_test, mnist_min, mnist_max,
fgsm_eps, is_fgsm=True)
print('')
evaluate_attack('mnist_fc_deepfool.csv', 'results', device, model,
mnist_loader_test, mnist_min, mnist_max, deep_args, is_fgsm=False)
if device.type == 'cuda': torch.cuda.empty_cache()

FGSM Test Error : 87.08%
FGSM Robustness : 1.56e-01
FGSM Time (All Images) : 0.15 s
FGSM Time (Per Image) : 14.99 us

DeepFool Test Error : 97.92%
DeepFool Robustness : 6.78e-02
DeepFool Time (All Images) : 141.81 s
DeepFool Time (Per Image) : 14.18 ms


**Исходя из результатов проверки атаки, можно сказать следующие:**
* 1 - Ошибка вносимая методом Deepfool - выше, чем FGSM
* 2 - Время, которое при этом затрачивается на одну картинку у метода DeepFool гораздо выше, т.к. данный метод доводит дисперсию шума до минимума
* 3 - Мера отвечающая за интенсивность шума (дисперсию шума) у метода DeepFool сильно ниже, что значит, что шум менее заметен, чем при FGSM